In [1]:
!pip install -q torch torchvision facenet-pytorch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.9 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
import random
import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from facenet_pytorch import InceptionResnetV1  # ArcFace backbone
import torch.nn.functional as F
from torch import nn, optim
from PIL import Image


In [3]:
class FacePairDataset(Dataset):
    def __init__(self, root_dir, transform=None, num_pairs_per_person=5):
        self.root_dir = root_dir
        self.transform = transform
        self.num_pairs_per_person = num_pairs_per_person
        self.image_paths = []  # All image paths
        self.labels = []  # Person IDs

        # Collect all images and their labels
        self.person_folders = sorted(os.listdir(root_dir))
        for idx, person in enumerate(self.person_folders):
            person_path = os.path.join(root_dir, person)
            images = [os.path.join(person_path, img) for img in os.listdir(person_path)]
            self.image_paths.extend(images)
            self.labels.extend([idx] * len(images))  # Assign numerical labels for each person

        # Create positive and negative pairs
        self.pairs = self.generate_pairs()

    def generate_pairs(self):
        pairs = []
        num_people = len(self.person_folders)

        for person_idx in range(num_people):
            # Get all images of a specific person
            person_images = [p for p, l in zip(self.image_paths, self.labels) if l == person_idx]

            # Positive pairs (same person)
            for _ in range(self.num_pairs_per_person):
                img1, img2 = random.sample(person_images, 2)
                pairs.append((img1, img2, 1))  # Label = 1 (Same Person)

            # Negative pairs (different people)
            for _ in range(self.num_pairs_per_person):
                img1 = random.choice(person_images)
                other_person_idx = random.choice([i for i in range(num_people) if i != person_idx])
                other_person_images = [p for p, l in zip(self.image_paths, self.labels) if l == other_person_idx]
                img2 = random.choice(other_person_images)
                pairs.append((img1, img2, 0))  # Label = 0 (Different People)

        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]

        img1 = Image.open(img1_path).convert("RGB")
        img2 = Image.open(img2_path).convert("RGB")

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)


In [6]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(p=0.5),  # Flips images to add variation
    transforms.RandomRotation(10),  # Rotates images by ±10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Adjusts lighting
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


# Load training and testing datasets
train_dataset = FacePairDataset(root_dir="/kaggle/input/faces1/archive/train", transform=transform)
test_dataset = FacePairDataset(root_dir="/kaggle/input/faces1/archive/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
class SiameseArcFace(nn.Module):
    def __init__(self):
        super(SiameseArcFace, self).__init__()
        self.arcface = InceptionResnetV1(pretrained='vggface2').eval()
        self.dropout = nn.Dropout(p=0.3)  # Helps prevent overfitting
        self.fc = nn.Linear(512, 1)

    def forward(self, img1, img2):
        emb1 = self.arcface(img1)
        emb2 = self.arcface(img2)
        distance = torch.abs(emb1 - emb2)
        distance = self.dropout(distance)  # Apply dropout before classification
        output = self.fc(distance)
        return output


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseArcFace().to(device)

# Loss function & optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary classification loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


# Training loop
num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    total_loss = 7

    for img1, img2, labels in train_loader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(img1, img2).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

print("Training Complete!")


  0%|          | 0.00/107M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/facenet_pytorch/models/inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)

Epoch [1/8], Loss: 0.9182
Epoch [2/8], Loss: 0.9114
Epoch [3/8], Loss: 0.9027
Epoch [4/8], Loss: 0.8917
Epoch [5/8], Loss: 0.8784
Epoch [6/8], Loss: 0.8576
Epoch [7/8], Loss: 0.8307
Epoch [8/8], Loss: 0.7927
Training Complete!


In [9]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for img1, img2, labels in dataloader:
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
            outputs = model(img1, img2).squeeze()
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy

train_acc = evaluate(model, train_loader)
test_acc = evaluate(model, test_loader)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


Train Accuracy: 0.9545
Test Accuracy: 0.8192


In [11]:
!pip install pickle-mixin

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5990 sha256=7f758d5c590bd6a08f031d1674d03d894673b64f18a13b83659aa93683c87815
  Stored in directory: /root/.cache/pip/wheels/3e/c6/e9/d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin


In [12]:
import pickle

with open("siamese_model.pkl", "wb") as f:
    pickle.dump(model, f)
